# **Agents Toolkit**

## **Librerías**

In [1]:
import sys
sys.path.append("../")

In [2]:
from scripts.basic_gen_text import basic_gen_text
from scripts.basic_openai_text import basic_openai
from scripts.basic_langchain_text import basic_langchain_text

In [3]:
from scripts.chat_roles import chat_roles
from scripts.chat_roles_langchain import chat_roles_langchain

In [4]:
from scripts.trim_messages import trim_messages_langchain

from scripts.prompt_template import prompt_template
from scripts.chat_prompt_template import chat_prompt_template
from scripts.few_shot_prompting import few_shot_prompt

In [ ]:
from scripts.chat_history_dummie import message_history
from scripts.streaming_chat import streaming_chat

In [10]:
from scripts.string_output_parser import st_out_parser
from scripts.runnable_lambda import runnable_lambda
from scripts.json_output_parser import json_parser

In [7]:
from warnings import filterwarnings
filterwarnings("ignore")

## **Agentes AI**

Un agente AI es una entidad que mediante el uso de un **modelo llm** como base, puede razonar, tomar decisiones e interactuar con su entorno para cumplir una objetivo especifico.

Para construir un agente AI basado en LLMs debemos entender como llamarlos y como utilizarlos de una manera óptima y eficiente dependiendo de la necesidad que tengamos.

### **Generar texto sencillo**

Podemos llamar modelos gratuitos desde plataformas como **HuggingFace**, sin embargo, la respuesta del modelo no es tan prometedora.

In [6]:
# Call a model from hugginface model id
basic_gen_text(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    prompt="Cuál es la capital de Colombia?",
    max_tokens=50
)

Device set to use cpu


'colombia'

Si queremos utilizar algo más robusto como los modelos actuales de openai, debemos crear una cuenta con ellos y cargar algo de saldo para poder utilizar su API, al hacer esto obtenemos respuestas mucho más interesantes.

In [7]:
basic_openai(
    model="gpt-3.5-turbo",
    prompt="Cuál es la capital de Colombia?",
    max_tokens=50
)

'La capital de Colombia es Bogotá.'

Incluso, con el uso de estos modelos podemos ir mucho más lejos.

In [10]:
basic_openai(
    model="gpt-3.5-turbo",
    prompt="Cuál crees que es el evento histórico más importante del siglo XXI en Colombia?",
    max_tokens=200
)

'Es difícil elegir un solo evento histórico como el más importante del siglo XXI en Colombia, ya que ha habido varios acontecimientos significativos que han impactado profundamente en el país. Algunos de los eventos más destacados podrían ser la firma del acuerdo de paz entre el gobierno colombiano y las FARC en 2016, el referéndum sobre el acuerdo de paz en el mismo año, la elección de Iván Duque como presidente en 2018, y la crisis migratoria y humanitaria en la frontera con Venezuela. Cada uno de estos eventos ha tenido un impacto significativo en la historia reciente de Colombia y ha marcado un antes y un después en la vida política y social del país.'

De igual forma, podemos realizar la conexión al modelo directamente desde **langchain**, una herramienta que nos permite interactuar con los **llms** de manera más sencilla.

In [11]:
basic_langchain_text(
    user_prompt="Qué es el universo?",
    model="gpt-4o"
)

'El universo es el conjunto total de espacio, tiempo, materia y energía que existe. Incluye todo lo que podemos observar y medir, así como lo que aún no hemos descubierto. El universo abarca galaxias, estrellas, planetas, materia oscura, energía oscura y cualquier otra forma de materia y energía existente. Se cree que se originó hace aproximadamente 13.8 mil millones de años con el evento conocido como el Big Bang, que marcó el comienzo de su expansión desde un estado extremadamente denso y caliente.\n\nEl universo está regido por las leyes de la física que conocemos, aunque todavía hay muchas incógnitas, especialmente relacionadas con la naturaleza de la materia oscura y la energía oscura, que constituyen la mayor parte de su contenido, y la estructura y destino último del universo. El estudio del universo, incluyendo su origen, evolución y destino, es el campo de la cosmología, una rama de la astronomía.'

Con esto ya podemos utilizar un LLM dentro de nuestro flujo pero para integrarlo al agente y sacarle el máximo provecho hay varias configuraciones adicionales que podemos tener en cuenta.

### **La definición de los roles**

Cuando pensamos en la interacción que tendremos con nuestro agente, la vía más común es a través de un chat. Este mecanismo nos permite mantener un flujo de conversación y dar retroalimentación activa sobre el desempeño que esta teniendo el agente.

De igual forma, con este tipo de interacción podemos aprovechar para agregar algunas configuraciones adicionales para darle ciertas instrucciones al modelo y aportarle contexto sobre lo que se espera de el.

En terminos generales hay 3 tipos de roles en este tipo de interacción:
- **system:** Es el rol que define el contexto y las reglas del sistema.
- **user:** Es el rol que define el usuario y sus acciones (Simulamos lo que podría decir el usuario).
- **assistant:** Es el rol que define el asistente y sus acciones. (Simulamos lo que podría decir el asistente).

In [12]:
chat_roles(
    model = "gpt-4o",
    system_prompt = "Eres un experto en libros de fantasía y ficción.",
    user_prompt = "Hola, ¿me puedes recomendar un libro poco conocido?"
)

'¡Por supuesto! Te recomiendo "La voz de las espadas" de Joe Abercrombie, el primer libro de la trilogía "La Primera Ley". Aunque Abercrombie ha ganado reconocimiento en los últimos años, esta serie a menudo queda eclipsada por otros gigantes del género. La novela combina una trama intrigante con personajes bien desarrollados y un toque de humor oscuro. Es ideal para lectores que disfrutan de la fantasía con un enfoque más realista y matizado.'

Esta asignación de rol tambien puede usarse con **langchain**.

In [13]:
chat_roles_langchain(
    model="gpt-4o",
    system_prompt="Eres un experto en libros de fantasía y ficción.",
    user_prompt="Hola, ¿me puedes recomendar un libro poco conocido?"
)

'¡Por supuesto! Te recomiendo "La voz de las espadas" de Joe Abercrombie. Aunque quizás no sea completamente desconocido, es una joya que a menudo pasa desapercibida en el género de la fantasía. Este libro es el primero de la trilogía "La Primera Ley" y se destaca por sus personajes complejos y una trama llena de giros inesperados, mezclando acción con toques de humor oscuro. Abercrombie tiene un estilo único que ofrece una visión más cínica y realista del típico mundo de fantasía. Si buscas una historia con personajes que desafían los arquetipos tradicionales, este libro podría ser justo lo que necesitas.'

### **Controlando el número de tokens**

De igual forma, es importante tener un control sobre el tamaño de los mensajes que estamos enviando a los modelos, por temas de costo y rendimientos. Existen varias estrategías para controlar esto.

In [14]:
# En este caso la estrategía consiste en conservar los últimos tokens enviados
trim_messages_langchain(
    model="gpt-4o",
    strategy="last",
    system_prompt="Eres un experto en libros de fantasía y ficción",
    user_prompt="Hola, ¿me puedes recomendar un libro poco conocido?"
)

'¡Por supuesto! Un libro de fantasía poco conocido pero que vale la pena explorar es "La ciudad y la ciudad" de China Miéville. Aunque Miéville es un autor reconocido en el género de la fantasía y la ficción especulativa, este libro en particular tiende a pasar desapercibido entre sus otros trabajos más famosos como "Perdido Street Station".\n\n"La ciudad y la ciudad" es una novela única que mezcla fantasía, ciencia ficción y elementos de novela policial. La historia transcurre en dos ciudades que ocupan el mismo espacio físico, pero cuyos habitantes están obligados a "no ver" a los de la otra ciudad debido a la severidad de sus leyes. El entramado cultural y político se explora a través de la investigación de un asesinato, lo que ofrece una narrativa cautivadora y profundamente original. \n\nSi te gusta el misterio con un toque de realismo mágico, este libro podría ser una excelente opción para ti.'

⚠️ **TAREA:** Revisar sobre métodos de control de tokens que resuman el historial.

### **Uso de Templates**

Siguiendo con las herramientas que nos suministra **langchain** para configurar mejor nuestros agentes, tenemos la opción de usar **templates**, que nos permiten definir una plantilla dinámica que podremos usar para personalizar el comportamiento del agente.

Con el uso de templates, podemos hacer que el código sea reutilizable y mantenible, aparte de eliminar redundancias.

In [6]:
# Ejemplo de uso
prompt_template('Hola, mundo!', 'frances')

'"Hola, mundo!" en francés se traduce como "Bonjour, le monde !"'

Existen templates especificos para interacciones de chat, en los que se puede simular una conversacion, que serviran para definir el contexto y la interaccion con el modelo.

In [20]:
chat_prompt_template(
    language='inglés',
    prompt='''
        Muchos años después, frente al pelotón de fusilamiento, 
        el coronel Aureliano Buendía había de recordar aquella 
        tarde remota en que su padre lo llevó a conocer el hielo. 
        Macondo era entonces una aldea de veinte casas de barro y
        cañabrava construidas a la orilla de un río de aguas diáfanas
        que se precipitaban por un lecho de piedras pulidas, 
        blancas y enormes como huevos prehistóricos'''
)

'Many years later, as he faced the firing squad, Colonel Aureliano Buendía was to remember that distant afternoon when his father took him to discover ice. Macondo was then a village of twenty mud and reed houses built on the bank of a river with clear waters that ran along a bed of polished stones, white and enormous like prehistoric eggs.'

Hay varios tipos de templates que podemos usar para robustecer a nuestros agentes. Uno de ellos es el few shot prompting, que permite al agente aprender de ejemplos anteriores.

In [8]:
prompt = 'Cuánto es 2 😎 9'

result = few_shot_prompt(prompt)
print(f'{prompt} -> {result}')

Cuánto es 2 😎 9 -> 18


### **Historial de Chat**

Uno de los aspectos más importantes a la hora de configurar un LLM es la capacidad que este tenga de entender el contexto de la conversación. Para ello, es importante mantener un historial que le permita recordar las distintas interacciones con el usuario. Aquí vuelve a tomar peso el uso de roles.

In [8]:
message_history()

¡Hola! ¿En qué puedo ayudarte hoy con respecto a la literatura?
--------------------
Parece que tu mensaje está vacío. Si tienes alguna pregunta o tema sobre literatura que te gustaría discutir, no dudes en decírmelo. Estoy aquí para ayudarte.
--------------------
¡Hola, Juan! Encantado de conocerte. ¿Hay algún tema literario o libro en particular sobre el que te gustaría hablar?
--------------------
Me dijiste que tu nombre es Juan. ¿En qué puedo ayudarte hoy?
--------------------
¡De nada, Juan! Si tienes más preguntas o necesitas ayuda con algo relacionado con la literatura, no dudes en decírmelo. Estoy aquí para ayudarte.
--------------------


### **Streaming**

Tambien tenemos una funcionalidad interesante que nos permite ver como se va generando la respuesta de nuestro prompt en tiempo real.

Este tipo de invocación funciona de manera asincrona.

In [18]:
await streaming_chat('Hola')

¡Hola! ¿En qué puedo ayudarte hoy?

### **Uso de Cadenas**

Ahora que conocemos varios elementos que nos permiten interactuar con nuestros agentes podemos empezar a combinarlos para crear lógicas más complejos. Para ello, se vuelve muy importante el uso de **cadenas.**

Las cadenas nos permiten conectar diferentes elementos de langchain para que se ejecuten en conjunto siguiendo un orden especifico.

In [ ]:
language = 'English'
prompt = '''
    La Rueda del tiempo gira y las eras llegan y pasan y dejan tras de sí
    recuerdos que se convierten en leyenda. La leyenda se difumina, deviene 
    mito, e incluso el mito se ha olvidado mucho antes de que la era que lo 
    vio nacer retorne de nuevo
'''

print(st_out_parser(prompt, language))

The Wheel of Time turns, and ages come and pass, leaving memories that become legend. Legend fades to myth, and even myth is long forgotten before the age that gave it birth comes again.


Dentro del uso de cadenas, tenemos la opción de usar bloques personalizados para definir el comportamiento esperado de nuestro agente.

In [8]:
runnable_lambda('¿Cuántos libros tiene la rueda del tiempo?')

{'Response': 'La serie "La Rueda del Tiempo" ("The Wheel of Time") consta de 14 libros principales, escritos por Robert Jordan, con los últimos tres completados por Brandon Sanderson tras el fallecimiento de Jordan. Además, hay una precuela titulada "Nueva Primavera" ("New Spring"). En total, son 15 libros si se cuenta la precuela.'}

Y lo más importante, podemos definir la manera en que queremos obtener la respuesta de nuestro agente, esto es muy importante cuando queramos configurar la conexión entre distintos servicios.

In [13]:
json_parser('Qué es el universo?')

{'definición': 'El universo es la totalidad del espacio, el tiempo, la materia y la energía. Incluye todas las galaxias, estrellas, planetas y cualquier otra forma de materia y energía, así como las leyes físicas que los gobiernan.',
 'componentes': ['Galaxias',
  'Estrellas',
  'Planetas',
  'Materia oscura',
  'Energía oscura',
  'Nebulosas',
  'Agujeros negros'],
 'origen': 'El universo se originó hace aproximadamente 13.8 mil millones de años con el evento conocido como el Big Bang, una explosión que marcó el inicio de su expansión.',
 'estructura': 'El universo observable tiene una estructura a gran escala que incluye cúmulos de galaxias, supercúmulos y filamentos, separados por grandes vacíos.',
 'expansión': 'El universo está en constante expansión, lo que significa que las galaxias se alejan unas de otras con el tiempo. Esta expansión se acelera debido a la energía oscura.'}